# FMM formulations

fmmax implements multiple _formulations_ of the FMM; the difference between formulations is in the method of generating _transverse permittivity matrices_ which relate the in-plane components of the electric field and the electric displacement field (in the Fourier representation). The simplest formulation is `FFT`, in which the blocks of the transverse permittivity matrix are simply the Fourier convolution matrices for their respective permittivity tensor components.

The remaining formulations are so-called _vector_ formulations, which make use of a vector field defined on the unit cell of the FMM calculation. At each point, the vector field defines a local coordinate system...

Multiple ways of using the fields to construct the transverse permittivity matrix... in fmmax, isotropic and anisotropic codepaths use two different methods. (Note that the anisotropic codepath can also be used for isotropic materials).


## Generating vector fields
In fmmax, vector fields are generated by solving an optimization problem where the objective is a function of the vector fields. In general, the objective rewards _smoothness_ and _alignment with a target field_, which is usually derived from the permittivity distribution in the unit cell. The objective is convex quadratic, but varies between formulations as discussed below. Also, the normalization of vector fields can depend on the formulation.

To keep the dimensionality of the problem low, we optimize the Fourier coefficients of the vector fields (rather than optimizing the fields on a real-space grid). This, together with the quadratic nature of the objective, means that a single Newton step can find the optimal vector field, with a cost that is low compared to other parts of an FMM calcualtion.

## Vector formulations

- `POL`: generates a complex linear vector field which has maximum magnitude 1 and a null in the interior of features. In the objective, the gradient of the vector field on the real-space grid is computed; a penalty term discourages non-smooth fields (i.e. fields whose gradient is large).
- `NORMAL`: takes the field computed by `POL`, but then normalizes so the magnitude is 1 evereywhere in the unit cell. Where `POL` has zeros, `NORMAL` has discontinuities.
- `JONES`: takes the field computed by `POL`, and converts it to an complex elliptical field which has magnitude 1 everywhere and lacks discontinuities.
- `JONES_DIRECT`: directly computes a complex elliptical vector field without first finding a linear vector field. Smoothness is evaluated on the real-space grid.
- `POL_FOURIER`: generates a complex linear vector field, but with an alternate method of penalizing non-smoothness. Specifically, the Fourier components corresponding to high spatial frequencies are penalized. Compared to `POL`, `POL_FOURIER` can be computed more efficiently.
- `NORMAL_FOURIER`: takes the field computed by `POL_FOURIER`, but then normalizes so the magnitude is 1 evereywhere in the unit cell.
- `JONES_FOURIER`: takes the field computed by `POL_FOURIER` and converts it to an complex elliptical field.
- `JONES_DIRECT_FOURIER`: directly computes a complex elliptical vector field, using Fourier coefficients to penalize non-smoothness.

## Properties of vector fields

The objective functions used by fmmax to obtain vector fields have been carefully chosen to produce fields with specfic characteristics. These include

- The fields are _independent of the unit cell scale_; if the unti cell is uniformlyy scaled (e.g. by changing the units from nanometers to microns), the vector fields are unchanged.
- The fields for a supercell (containing e.g. 2x2 unit cells) are identical to those of a single unit cell, so long as the number of terms in the Fourier expansion is increased correspondingly. Note that this means that  the tangent field depends upon the number of terms in the Fourier expansion.
- The tangent field is independent of the resolution of the discretized unit cell. That is, whether the permittivity distribution is specified with a `(100, 100)` or `(200, 200)` shaped array has no impact on the resulting field.
- The tangent field is independent of the amplitude of the array from which it is obtained, e.g. the permittivity contrast.

[1] V. Liu and S. Fan, “S4 : A free electromagnetic solver for layered periodic structures,” Comput. Phys. Commun. 183(10), 2233–2244 (2012).
